In [1]:
%pip install os
%pip install pydoc
%pip install faiss-cpu --no-cache  # Use 'faiss-gpu' for GPU version
%pip install io
%pip install environs
%pip install moviepy
%pip install Django
%pip install langchain
%pip install openai

import os
from pydoc import cli
import faiss
import io
from environs import Env
from moviepy.editor import *
from moviepy.editor import VideoFileClip
from django.core.files.storage import default_storage
from django.core.files.base import ContentFile


from langchain import PromptTemplate
from langchain import OpenAI
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQAWithSourcesChain



ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement pydoc (from versions: none)
ERROR: No matching distribution found for pydoc
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 720.7 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
Note: you may need to restart the kernel to use updated packages.
  Using cached environs-11.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached marshmallow-3.21.2-py3-none-any.whl.metadata (7.1 kB)
Using cached environs-11.0.0-py3-none-any.whl (12 kB)
Using cached marshmallow-3.21.2-py3-none-any.whl (49 kB

In [2]:
from environs import Env

env = Env()
# Read .env into os.environ

env.read_env()
openai.api_key = os.environ.get("OPENAI_API_KEY")



OSError: Starting path not found

In [3]:

# Main application
def main_processor(file_obj, video_uuid):

    d = dict()
    # d['title'] ='What is The Curse of Dimensionality?'
    # d['desc'] = 'This video explains the concept of Curse of Dimensionality in Machine Learning. This is coming from Dummy Data in VideoProcessors module, please re-activate the default OpenAI api to get real results.'
    # d['tags'] = 'Machine Learning, Data, Models'

    videoclip = VideoFileClip(file_obj.temporary_file_path())

    # print(videoclip.audio)
    if not videoclip.audio:
        d["transcript"] = ""
        d["title"] = ""
        d["desc"] = ""
        d["tags"] = ""
    else:
        videoclip.audio.write_audiofile("temp.mp3")

        # Transcripe MP3 Audio function
        with open("temp.mp3", "rb") as audio_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
            d["transcript"] = transcript.text

        # Delete the original audio file
        os.remove("temp.mp3")

        # Splitting the text
        textsplitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)

        texts = textsplitter.split_text(transcript.text)
        # d['transcript'] = texts

        store = FAISS.from_texts(
            texts,
            OpenAIEmbeddings(),
            metadatas=[
                {"source": f"Text chunk {i} of {len(texts)}"} for i in range(len(texts))
            ],
        )

        faiss.write_index(store.index, "docs.faiss")

        llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
        chain = RetrievalQAWithSourcesChain.from_chain_type(
            llm=llm, chain_type="stuff", retriever=store.as_retriever()
        )

        # AI assisted title
        question = "Come up with a short title under 100 characters that best summarize this text! Make the title as informative as possible!"
        answer = chain({"question": question})
        # Remove quotes from the title
        d["title"] = answer["answer"].strip('" \n')

        # AI assisted short description
        question = "Summarize the transcript under 160 characters! Make the description as informative as possible!"
        answer = chain({"question": question})
        d["desc"] = answer["answer"].strip('" \n')
        # AI assisted tags
        question = "Come up with the top five keywords that summarize this text! In your response only present the top five keywords separated by commas, nothing else."
        answer = chain({"question": question})
        d["tags"] = answer["answer"].strip('" \n')

    return d


def validate_video(file_obj, video_uuid):
    try:

        # Handle temporary file
        clip = VideoFileClip(file_obj.temporary_file_path())

        # Rest of your validation logic
        if not file_obj.name.endswith(".mp4"):
            return False, "Invalid format"

        if clip.duration > 180:
            return False, "Duration exceeds limit"

        width, height = clip.size
        video_type = "landscape" if width > height else "portrait"

        # Check that the video's resolution is at least 720p
        if (video_type == "landscape" and (width < 1280 or height < 720)) or (
            video_type == "portrait" and (width < 720 or height < 1280)
        ):
            return False, "Video resolution must be at least 720p"

        clip.save_frame(f"{video_uuid}.png", t="00:00:01")

        return True, video_type

    except Exception as e:
        print(f"An error occurred while validating the video: {e}")
        return False, str(e)

    finally:
        clip.close() if "clip" in locals() else None

In [4]:
def main_processor(mp4_file_path, video_uuid):
    d = dict()

    videoclip = VideoFileClip(mp4_file_path)

    if not videoclip.audio:
        d["transcript"] = ""
        d["title"] = ""
        d["desc"] = ""
        d["tags"] = ""
    else:
        videoclip.audio.write_audiofile("temp.mp3")

        # Transcribe MP3 Audio function
        with open("temp.mp3", "rb") as audio_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
            d["transcript"] = transcript.text

        # Delete the original audio file
        os.remove("temp.mp3")

        # Splitting the text
        textsplitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)

        texts = textsplitter.split_text(transcript.text)

        store = FAISS.from_texts(
            texts,
            OpenAIEmbeddings(),
            metadatas=[
                {"source": f"Text chunk {i} of {len(texts)}"} for i in range(len(texts))
            ],
        )

        faiss.write_index(store.index, "docs.faiss")

        llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
        chain = RetrievalQAWithSourcesChain.from_chain_type(
            llm=llm, chain_type="stuff", retriever=store.as_retriever()
        )

        # AI-assisted title, description, and tags
        question = "Provide a short title summarizing this transcript, followed by a brief description, and then list the top five keywords. Separate each part using '#;)' (e.g., Title#;) Description#;) Keywords)."
        answer = chain({"question": question})
        response = answer["answer"].strip()

        # Separate the response by "#;)"
        split_response = response.split("#;)")

        # Assign the results into the dictionary
        d["title"] = split_response[0].strip()
        d["desc"] = split_response[1].strip()
        d["tags"] = split_response[2].strip()

    return d


In [7]:
main_processor("test.mp4","123")

NameError: name 'VideoFileClip' is not defined